In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [2]:
file_path = "Resources/cleaned_merged_data.csv"
data = pd.read_csv(file_path)

In [3]:
trust_metrics = [
    "doctor_and_nurses", "government", "journalist",
    "neighbourhood", "philantropist", "scientist", "traditional_healers"
]

## First scatter plot showcasing global trust in Government by Country

In [4]:
# world map
fig = go.Figure()

# added a scattergeo layer for government trust rates
fig.add_trace(go.Scattergeo(
    lat=data['Latitude'],
    lon=data['Longitude'],
    text=data['country'] + "<br>Trust in Government: " + data['government'].astype(str) + "%",
    mode='markers',
    marker=dict(
        size=10,  
        color=data['government'], 
        colorscale='hot',
        colorbar_title="Trust in Government (%)",
        line=dict(width=0.5, color='black'),
    )
))

# updated the map layout
fig.update_layout(
    title='Global Trust in Government by Country',
    geo=dict(
        showframe=False,
        showcoastlines=True,
        coastlinecolor="gray",
        showland=True,
        landcolor="lightgray",
        bgcolor="lightblue",
        projection_type='natural earth',
        showcountries=True,
        countrycolor="black",  
        countrywidth=0.3
    ),
    template='plotly',
    height=800,
    width=1200
)

fig.show()

In [5]:
trust_categories = {
    "government": "hot",
    "scientist": "Viridis",
    "journalist": "Plotly3",
    "doctor_and_nurses": "Plasma",
    "philantropist": "Inferno",
    "traditional_healers": "Turbo",
    "neighbourhood": "Magma"
}

In [6]:
min_marker_size = 5
max_marker_size = 45
data['Population scaled'] = data['population'].apply(lambda x: np.log1p(x))  # Logarithmic scale
data['Population scaled'] = ((data['Population scaled'] - data['Population scaled'].min()) /
                             (data['Population scaled'].max() - data['Population scaled'].min()) *
                             (max_marker_size - min_marker_size)) + min_marker_size


fig = go.Figure()

In [7]:
for i, (category, colorscale) in enumerate(trust_categories.items()):
    fig.add_trace(go.Scattergeo(
        lat=data['Latitude'],
        lon=data['Longitude'],
        text=data['country'] + f"<br>Trust in {category.capitalize()}: " + data[category].astype(str) + "%" +
             "<br>Population: " + data['population'].apply(lambda x: f"{x:,}"),
        mode='markers',
        marker=dict(
            size=data['Population scaled'],
            color=data[category],
            colorscale=colorscale,
            colorbar=dict(
                title=f"Trust in {category.capitalize()} (%)",
                len=0.5,
                thickness=15,
                x=1.1,
                xpad=10
            ),
            line=dict(width=0.5, color='black')
        ),
        name=category.capitalize(),
        visible=(i == 0) 
    ))

## Interactive Visualization of every trust category sorted by user input

In [8]:
fig.update_layout(
    title=dict(
        text="Trust in Government by Country",
        font=dict(size=20),
        x=0.5,
        xanchor="center"
    ),
    updatemenus=[
        dict(
            buttons=[
                dict(
                    label=category.capitalize(),
                    method="update",
                    args=[
                        {"visible": [i == j for i in range(len(trust_categories))]},  
                        {"title.text": f"Trust in {category.capitalize()} by Country"}
                    ]
                ) for j, category in enumerate(trust_categories)
            ],
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.2,
            yanchor="top"
        )
    ],
    geo=dict(
        showframe=False,
        showcoastlines=True,
        coastlinecolor="gray",
        showland=True,
        landcolor="lightgray",
        bgcolor="lightblue",
        projection_type='natural earth',
        showcountries=True,
        countrycolor="black",
        countrywidth=1.5
    ),
    legend=dict(
        x=-0.2,
        y=0.9,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.5)",
        bordercolor="black",
        borderwidth=1
    ),
    template='plotly',
    height=600,
    width=1000
)

fig.show()